### Introduction
This notebook contains a simple GUI constructing using Jupyter Widgets intended to help with testing, demonstrate the functionality to non-technical users, and act as an example of using the Python API to technical users. It is not intended as a production usage GUI and the Python code behind the GUI is not of production quality and robustness.

Each input tab of the GUI contains buttons to save/load inputs to/from CSV files in order to avoid repeatedly typing in the same data each time th GUI is launched. Some example of such persisted data can be found in the directory ./storage_gui_data/ in the same directory as this notebook.

After installing the prerequisites listed below, simply run the single cell and the GUI will appear.

#### Prerequisites
This notebook requires an experimental version of the cmdty-storage package only published on Test PyPI, plus the packages curves for smoothing the forward curve, and ipysheet which provides the input and output grids.

`
pip install -i https://test.pypi.org/simple/ cmdty-storage==0.1.0a17
pip install curves
pip install ipysheet
`

If this notebook has been downloaded standalone, i.e. without downloading the whole storage repo,
it requires the file [storage_gui.py](https://github.com/cmdty/storage/blob/master/samples/python/storage_gui.py) be in the same directory.


In [1]:
%matplotlib inline
import storage_gui as sg
sg.display_gui()